In [42]:
!pip install kaggle

In [43]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [44]:
!kaggle datasets download -d promptcloud/walmart-product-review-dataset

Dataset URL: https://www.kaggle.com/datasets/promptcloud/walmart-product-review-dataset
License(s): CC0-1.0
walmart-product-review-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [45]:
from zipfile import ZipFile

dataset = '/content/walmart-product-review-dataset.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import os
from scipy.sparse import coo_matrix

In [47]:
df= pd.read_csv('/content/marketing_sample_for_walmart_com-walmart_com_product_review__20200701_20201231__5k_data.tsv', sep='\t')
df.columns

Index(['Uniq Id', 'Crawl Timestamp', 'Dataset Origin', 'Product Id',
       'Product Barcode', 'Product Company Type Source',
       'Product Brand Source', 'Product Brand Normalised Source',
       'Product Name Source', 'Match Rank', 'Match Score', 'Match Type',
       'Retailer', 'Product Category', 'Product Brand', 'Product Name',
       'Product Price', 'Sku', 'Upc', 'Product Url', 'Market',
       'Product Description', 'Product Currency',
       'Product Available Inventory', 'Product Image Url',
       'Product Model Number', 'Product Tags', 'Product Contents',
       'Product Rating', 'Product Reviews Count', 'Bsr', 'Joining Key'],
      dtype='object')

In [48]:
df.head(5)

,Uniq Id,Crawl Timestamp,Dataset Origin,Product Id,Product Barcode,Product Company Type Source,Product Brand Source,Product Brand Normalised Source,Product Name Source,Match Rank,...,Product Currency,Product Available Inventory,Product Image Url,Product Model Number,Product Tags,Product Contents,Product Rating,Product Reviews Count,Bsr,Joining Key
0,1705736792d82aa2f2d3caf1c07c53f4,2020-09-24 03:21:12 +0000,NaN,2e17bf4acecdece67fc00f07ad62c910,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/0e1f4c51-c1a4-4f38-91bb-fd19317e3e20.aae056ad1dcaab1abc7f13c21ca53440.jpeg,NaN,"OPI Infinite Shine, Nail Lacquer Nail Polish, Bubble Bath, Wal-mart, Walmart.com",NaN,NaN,NaN,NaN,81350af1be98d3753cf964709f0c766a
1,95a9fe6f4810fcfc7ff244fd06784f11,2020-10-30 14:04:08 +0000,NaN,076e5854a62dd283c253d6bae415af1f,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/9c8e42e4-13a5-4b7e-aa1b-3af15267c55c_1.5e1a4b5e7161c6c26008e5c8f1569d7c.jpeg,NaN,"Nice 'n Easy Permanent Color, 111 Natural Medium Auburn 1 ea (Pack of 3), Wal-mart, Walmart.com",NaN,NaN,NaN,NaN,0353e63907dc0de0c734db4690300057
2,8d4d0330178d3ed181b15a4102b287f2,2020-08-06 05:51:47 +0000,NaN,8a4fe5d9c7a6ed26cc44d785a454b124,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/e3a601c2-6a2b-4cc8-bb35-a9049930ded9_1.ab14e053feea88a9026baeca4d49cb77.jpeg | https://i5.walmartimages.com/asr/0486808b-9ce5-4df5-bf1e-da93f1dcbf83_1.dda3e2ede735f709cf380158706b30c5.jpeg | https://i5.walmartimages.com/asr/430f222f-7889-4016-836d-d82a9c4828b0_1.489791f211ea0d0b7afd6bdb49e4334a.jpeg | https://i5.walmartimages.com/asr/56b9720a-542d-4659-b96c-1b21538e841c_1.2d25a3568680a10f9fc13ec3bdb55936.jpeg | https://i5.walmartimages.com/asr/765046f4-f3f9-406d-8544-5f99dbc1b508_1.4bd398150d394b8480c4d6052b130b73.jpeg | https://i5.walmartimages.com/asr/2e393d46-7b44-41ed-afa2-dc1869f2500f_1.80b12f897c3321b65293529848062622.jpeg | https://i5.walmartimages.com/asr/757b7b18-c2a9-4677-bd5a-661175983ba1_1.219bca851bc5c1aa6d27067579a94d7b.jpeg,NaN,"Clairol Nice 'N Easy Permanent Color 7/106A Natural Dark Neutral Blonde, 1.0 KIT, Wal-mart, Walmart.com",NaN,4.5,29221.0,NaN,b6985c8e94815fbca2319dbb8bf228af
3,fddc4df45b35efd886794b261f730c51,2020-07-15 11:22:04 +0000,NaN,03b5fb878a33eadff8b033419eab9669,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/25b4b467-bc61-4e59-ba14-a123866e3384_1.453b236381c1d21a20c646821a47da0a.jpeg,NaN,"Kokie Professional Matte Lipstick, Hot Berry, 0.14 fl oz, Wal-mart, Walmart.com",NaN,NaN,NaN,NaN,85b70fded09186f00467cea2f935b779
4,0990cf89a59ca6a0460349a3e4f51d42,2020-11-26T12:27:20+00:00,NaN,ce3d761e57d6ccad80619297b5b1bcbc,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/1a2ebb06-cd01-4c73-9c6a-28a9c5ff8c1d.49cd30de37f9863bc4b217384bdedbcc.jpeg | https://i5.walmartimages.com/asr/3201391a-02cd-44b4-9112-641e407b8333.4224c67e1423877e778c613cfc7695bc.jpeg | https://i5.walmartimages.com/asr/7d5f5581-1fa2-44f5-878b-9b07da3bcdab.f38f6a90c05b25f2acb3993f288096a0.jpeg | https://i5.walmartimages.com/asr/38f166ea-3ef0-4a80-97cd-390f3830618e.5c9b0fde0c76990a300846b3e4ccb506.jpeg | https://i5.walmartimages.com/asr/44337d82-c7fb-476e-85e3-34c6d198922f.0c84ac688d94f8d7df8be9299a5d70c5.jpeg | https://i5.walmartimages.com/asr/8c250c82-4c32-4afe-9541-a4791f333a90.a0ad3d2c46475242722732c4c79692c3.jpeg | https://i5.walmartimages.com/asr/e6e0df7a-2ce0-4ba1-809e-6bc5d26e3a9d.c08f23e98f2a55897c1698357004146c.jpeg,NaN,"Gillette TRAC II Plus Razor Blade Refills, Fit TRAC II Handles, 10 ct, Wal-mart, Walmart.com",NaN,NaN,131.0,NaN,41c870871328e97da6fb036bb7d4b2da


In [49]:
df.shape

(5000, 32)

In [50]:
df.isnull().sum()

,0
Uniq Id,0
Crawl Timestamp,0
Dataset Origin,5000
Product Id,0
Product Barcode,5000
Product Company Type Source,0
Product Brand Source,4861
Product Brand Normalised Source,4861
Product Name Source,4861
Match Rank,5000


In [51]:
df = df[['Uniq Id','Product Id', 'Product Rating', 'Product Reviews Count', 'Product Category', 'Product Brand', 'Product Name', 'Product Image Url', 'Product Description', 'Product Tags']]
df.head(5)

,Uniq Id,Product Id,Product Rating,Product Reviews Count,Product Category,Product Brand,Product Name,Product Image Url,Product Description,Product Tags
0,1705736792d82aa2f2d3caf1c07c53f4,2e17bf4acecdece67fc00f07ad62c910,NaN,NaN,Premium Beauty > Premium Makeup > Premium Nail Polish & Care > Premium Nail Polish,OPI,"OPI Infinite Shine, Nail Lacquer Nail Polish, Bubble Bath",https://i5.walmartimages.com/asr/0e1f4c51-c1a4-4f38-91bb-fd19317e3e20.aae056ad1dcaab1abc7f13c21ca53440.jpeg,NaN,"OPI Infinite Shine, Nail Lacquer Nail Polish, Bubble Bath, Wal-mart, Walmart.com"
1,95a9fe6f4810fcfc7ff244fd06784f11,076e5854a62dd283c253d6bae415af1f,NaN,NaN,Beauty > Hair Care > Hair Color > Auburn Hair Color,Nice'n Easy,"Nice n Easy Permanent Color, 111 Natural Medium Auburn 1 ea (Pack of 3)",https://i5.walmartimages.com/asr/9c8e42e4-13a5-4b7e-aa1b-3af15267c55c_1.5e1a4b5e7161c6c26008e5c8f1569d7c.jpeg,"Pack of 3 Pack of 3 for the UPC: 381519000201 Beautiful, natural color. Active Ingredients: Color-Blend Formula: Water; Ethoxydiglycol; Propylene Glycol; Isopropyl Alcohol; Soytrimonium Chloride; Oleth-5; Oleic Acid; Ammonium Hydroxide; Trisodium Ethylenediamine Disuccinate; Fragrance; Oleth-2; C11-15 Pareth-9; Cocamidopropyl Betaine; C12-15 Pareth-3; Erythorbic Acid; Citric Acid; Sodium Sulfite; EDTA; p-Aminophenol; 4-Amino-2-Hydroxytoluene; p-Phenylenediamine; Resorcinol; 2-Methylresorcinol; N,N-Bis(2-Hydroxyethyl)-P-Phenylenediamine Sulfate; 1-Hydroxyethyl 4,5-Diamino Pyrazole Sulfate; Phenyl Methyl Pyrazolone; 1-Naphthol. Color-Blend Activator: Water; Hydrogen Peroxide; Acrylates Copolymer; Steareth-21; Oleth-2; Oleth-5; PEG-50 Hydrogenated Palmamide; Acrylates/Steareth-20 Methacrylate Copolymer; Oleyl Alcohol; Etidronic Acid; Disodium EDTA; Simethicone. Colorseal Conditioning Gloss: Water; Amodimethicone; Stearyl Alcohol; Cetyl Alcohol; Stearamidopropyl Dimethylamine; Glutamic Acid; Fragrance; Benzyl Alcohol; Panthenyl Ethyl Ether; EDTA; Panthenol; Trimethylsiloxysilicate Carthamus Tinctorius (Safflower) Seed Oil; Cocos Nucifera (Coconut) Oil; Hydrolyzed Sweet Almond Protein; Methylchloroisothiazolinone; Methylisothiazolinone; Aloe Barbadensis Leaf Uses: Salon experts know it takes a perfect blend of tones to create amazingly natural color results. Nice n Easy creates beautifully blended natural-looking shades for you, just like the experts, in one easy step! Q. How do I know if Nice n Easy is right for me? A. Nice n Easy gives you ultimate natural-looking color with highlights and shine in just one easy step. Browns are brilliant, blondes are illuminating, and reds are radiant. Q. How does it work? A. Only Nice n Easy has our exclusive Color-blend formula. In one step, each shade gives you an expert blend of 3 tones that work together to create natural-looking color with shining dimension. Q. Will it cover my grays? A. Yes! Nice n Easy promises 100% gray coverage even on resistant grays. Q. How long will it last? A. Nice n Easy lasts for up to 6 weeks. And for your radiant red shade, youll find 6 weekly treatments of our exclusive ColorSeal Conditioning Gloss to seal in healthy-looking color and add intense shine.","Nice 'n Easy Permanent Color, 111 Natural Medium Auburn 1 ea (Pack of 3), Wal-mart, Walmart.com"
2,8d4d0330178d3ed181b15a4102b287f2,8a4fe5d9c7a6ed26cc44d785a454b124,4.5,29221.0,Beauty > Hair Care > Hair Color > Permanent Hair Color,Clairol,"Clairol Nice N Easy Permanent Color 7/106A Natural Dark Neutral Blonde, 1.0 KIT",https://i5.walmartimages.com/asr/e3a601c2-6a2b-4cc8-bb35-a9049930ded9_1.ab14e053feea88a9026baeca4d49cb77.jpeg | https://i5.walmartimages.com/asr/0486808b-9ce5-4df5-bf1e-da93f1dcbf83_1.dda3e2ede735f709cf380158706b30c5.jpeg | https://i5.walmartimages.com/asr/430f222f-7889-4016-836d-d82a9c4828b0_1.489791f211ea0d0b7afd6bdb49e4334a.jpeg | https://i5.walmartimages.com/asr/56b9720a-542d-4659-b96c-1b21538e841c_1.2d25a3568680a10f9fc13ec3bdb55936.jpeg | https://i5.walmartimages.com/asr/765046f4-f3f9-406d-8544-5f99dbc1b508_1.4

In [52]:
df.isnull().sum()

,0
Uniq Id,0
Product Id,0
Product Rating,2806
Product Reviews Count,1654
Product Category,10
Product Brand,13
Product Name,0
Product Image Url,0
Product Description,1127
Product Tags,0


In [53]:
df['Product Rating'].fillna(0, inplace=True)
df['Product Reviews Count'].fillna(0, inplace=True)
df['Product Category'].fillna('', inplace=True)
df['Product Brand'].fillna('', inplace=True)
df['Product Description'].fillna('', inplace=True)

In [54]:
df.isnull().sum()

,0
Uniq Id,0
Product Id,0
Product Rating,0
Product Reviews Count,0
Product Category,0
Product Brand,0
Product Name,0
Product Image Url,0
Product Description,0
Product Tags,0


In [55]:
column_name_mapping = {
    'Uniq Id': 'ID',
    'Product Id': 'ProdID',
    'Product Rating': 'Rating',
    'Product Reviews Count': 'ReviewCount',
    'Product Category': 'Category',
    'Product Brand': 'Brand',
    'Product Name': 'Name',
    'Product Image Url': 'ImageURL',
    'Product Description': 'Description',
    'Product Tags': 'Tags',
    'Product Contents': 'Contents'
}
df.rename(columns=column_name_mapping, inplace=True)

In [56]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def clean_and_extract_tags(text):
    doc = nlp(text.lower())
    tags = [token.text for token in doc if token.text.isalnum() and token.text not in STOP_WORDS]
    return ', '.join(tags)

columns_to_extract_tags_from = ['Category', 'Brand', 'Description']
for column in columns_to_extract_tags_from:
    df[column] = df[column].apply(clean_and_extract_tags)

In [57]:
df['Tags'] = df[columns_to_extract_tags_from].apply(lambda row: ', '.join(row), axis=1)

Rating based recommendation system

In [58]:
average_ratings = df.groupby(['Name','ReviewCount','Brand','ImageURL'])['Rating'].mean().reset_index()
average_ratings

,Name,ReviewCount,Brand,ImageURL,Rating
0,$6 Mail-In Rebate Available Braun Epilator Silk-epil 3-270 with Bikini Trimmer and 3 Extras,146.0,braun,https://i5.walmartimages.com/asr/c7d78838-e8e2-416d-b7bc-c20fd9e2afbf.6868296b620c565430aac9d1b5b93fe0.jpeg | https://i5.walmartimages.com/asr/a8db8801-0a66-4a0b-acdf-d2c95b6d3b9a.6bb1250c842e85d9dd8ceb26361cf80b.jpeg | https://i5.walmartimages.com/asr/98e6f5c3-043d-467d-a64d-dfa0443cec31.01a43c1db5dbe1e5e8c0be1358a0416b.jpeg | https://i5.walmartimages.com/asr/5b7c55d2-02ee-4ee4-b25e-e0231720463e.7775eb4d90e863e2868d532aac6889be.jpeg | https://i5.walmartimages.com/asr/579572c7-b914-4a78-b5c0-37335226267f.c981f0fdbf69e986e84398b5e48eba43.jpeg | https://i5.walmartimages.com/asr/783fd3c7-61fd-4b02-8098-3d37cc212450.5456f1ef2060bb92041b085a56d3380f.jpeg | https://i5.walmartimages.com/asr/29b589ab-b05c-4625-83c3-7f178a1c163a.ce365d21b729256c100a52c5d3b564a2.jpeg,4.5
1,($100 Value) Peter Thomas Roth Thermal Therapy Duo Skincare Kit,1.0,"peter, thomas, roth",https://i5.walmartimages.com/asr/ecd851ef-6ca7-4fa8-a2e2-e8ef008a3096.1eec4055ad879f5626c2e9a75f07c9ab.jpeg | https://i5.walmartimages.com/asr/18483785-2716-41ab-89e6-f11ca02450b5_1.4340cf7899537b0098029eee6e3970b3.jpeg,5.0
2,($11 Value) Love Beauty and Planet Coconut Water Mimosa & Lavender Argan Oil Hand Cream Holiday Gift Set 2 Ct,278.0,"love, beauty, planet",https://i5.walmartimages.com/asr/0348d82d-2a13-4092-846d-fafaf8663293.c49e7b4158a671cf43e38b1de5d5a953.jpeg | https://i5.walmartimages.com/asr/a8888f60-fb79-45d2-810a-9684babe9791.990f704b0c047fd225fb1598df5bfc15.jpeg | https://i5.walmartimages.com/asr/1cdec1fc-045e-4e87-94ea-191147300c85.03af6453246961afc6d4239edde6306f.jpeg | https://i5.walmartimages.com/asr/dd61401a-e229-44dd-9a12-1303c3f7e83f.d78deb63c8399307ac55eb7f51a8f236.jpeg | https://i5.walmartimages.com/asr/6de167d7-dfbb-4662-9da4-15cd7542a95b.fbd0cf6d295b6ab8aa27fe88f39adaf1.jpeg,0.0
3,"($125 Value) Impressions Vanity Touch Pro LED Makeup Mirror with Wireless Bluetooth Audio & USB Charger, Rose Gold",2.0,"impressions, vanity",https://i5.walmartimages.com/asr/b886b154-97fd-4657-a122-fc3308f96ea2_1.0ab48e97667446ecff4ec73679a1ee47.jpeg | https://i5.walmartimages.com/asr/16acedba-0147-49b8-a4d0-4f59f8f1c0e4_1.4546a5261bd610f324b94d738635340b.jpeg,2.5
4,"($129 Value) CHI G2 Ceramic Titanium Infused Hairstyling Flat Iron Straightener, 1&quot",23.0,chi,https://i5.walmartimages.com/asr/9bcc0ed7-d62c-4a1d-99d2-8946e7b8b485_1.5185d1bcc29219b9176fc6b0c3be4538.jpeg | https://i5.walmartimages.com/asr/80e881b2-ccc7-46ab-83a8-ac64ba5a91f9_1.07e514f92c5d7ab964349cf6607a165d.jpeg | https://i5.walmartimages.com/asr/d79f1018-878b-4e3e-902c-23ffe56bc5b7_1.56af9c84c637dc7b943a94236226c37f.jpeg,4.4
...,...,...,...,...,...
4987,"wet n wild Photo Focus Correcting Palette, Color Commentary",12.0,"wet, n, wild",https://i5.walmartimages.com/asr/750275fb-ccfa-41d8-9604-f07dc782a56c.2830c8863b1db7c62287efc3284c4c7f.jpeg | https://i5.walmartimages.com/asr/c582fb64-1cb8-41d3-9738-a8b4568d7d23.70d25abd961e14afa3473dc8cc33ebf8.jpeg | https://i5.walmartimages.com/asr/cab44a31-8933-4fe1-adb0-b71b607c3593.a3dd51fe22c2ff50d49dc625a86b1a41.jpeg,3.3
4988,"wet n wild Photo Focus Liquid Foundation, Medium Dark, Matte, Desert Beige, 1 fl oz",7827.0,"wet, n, wild",https://i5.walmartimages.com/asr/c2c5dc7e-9bbb-43ef-b643-a2ae0dd0c901.61c580045c5c5809b5cc7408b4e52702.jpeg | https://i5.walmartimages.com/asr/fe1f9ec0-6c6e-44cd-b576-d53f4d13cdef.c80612867ea330eca1a4b93c8513ebb7.jpeg | https://i5.walmartimages.com/asr/23eed268-989c-446b-b22c-4470b04e5ad2.0912d256d830d82d1c959ecbf631fb3a.jpeg | https://i5.walmartimages.com/asr/4c8cca44-e43e-4d2a-bf29-02873eec283e.87eb914d0935d6c3c411d192ad00d675.jpeg,0.0
4989,"wet n wild Photo Focus Pressed Powder, Dark Café",635.0,"wet, n, wild",https://i5.walmartimages.com/asr/57bd497f-0d5c-44d1-8039-2f1f5b387b52.cd61296b9f5861c3528bbe655fd6acf0.jpeg | https://i5.walmartimages.com/asr/e79585ec-0185-4791-a753-ff864ed9b118.d7de8b3d

In [59]:
top_rated_items = average_ratings.sort_values(by='Rating', ascending=False)

rating_base_recommendation = top_rated_items.head(10)

In [60]:
rating_base_recommendation

,Name,ReviewCount,Brand,ImageURL,Rating
1686,"Electric Shaver, Triple Shaving Time Electric Rotary Razor for Wet Dry Shaving Waterproof Cordless with Pop-up Trimmer USB Rechargeable",4.0,moosoo,https://i5.walmartimages.com/asr/e7dcd553-90df-4cf8-8509-bc8035673838.f4f6365105fe9eb0af1bccab5d54b212.jpeg | https://i5.walmartimages.com/asr/5eb0db2d-d3cf-48ba-9017-67bf26d2600f.8d0dd42287deaf6d2b999aa842161d0e.jpeg | https://i5.walmartimages.com/asr/49f1fbb4-b36d-4032-bdf2-9d8402d5ec4d.14532a165720504c8c1d102920bced28.jpeg | https://i5.walmartimages.com/asr/70c133b3-8386-4322-9513-315b33ec21b4.f52654b1dcdd657c6fc6b505c3783a71.jpeg | https://i5.walmartimages.com/asr/b5a354b4-6b28-4cf9-95c5-13a95ea05a62.07b544b8fb25ad4f835d91e69a85daa1.jpeg | https://i5.walmartimages.com/asr/971d2944-630e-4f01-b854-64c121d77973.2edef59cf6780c2f978a33806315662e.jpeg | https://i5.walmartimages.com/asr/80b8528c-08a6-44bd-af75-3aaaa82ed233.d9b9c9a273ad3fe373e15ac16fab5a9d.jpeg | https://i5.walmartimages.com/asr/c4ef66b8-bc76-482e-9a01-82a5317b59be.1ba2aa4f09cf81b14047d6e88b73f51d.jpeg | https://i5.walmartimages.com/asr/6fe4642d-1c1e-4d93-977d-93db17bc37b4.ce7bcefc6486ad1739613c9948c70382.png | https://i5.walmartimages.com/asr/1574b3de-51ba-4146-b65d-c42df0f4478e.5da6e7b5df73f83c7303a25ff2bde2de.jpeg,5.0
526,"Alaffia Body Lotion, Vanilla, 32 Oz",2.0,alaffia,https://i5.walmartimages.com/asr/2988c323-cb6f-4a45-9bd7-9029d981630c_1.d65b6410f1b5a72233cdab07e25e153b.jpeg | https://i5.walmartimages.com/asr/c773f069-a99d-4d06-a4ff-d4c839063929_1.3a72dc9092da4efd57530dc86a077c6d.jpeg,5.0
2053,"Gold Bond Ultimate Ultimate Healing Lotion, Aloe 1 oz",2.0,"gold, bond",https://i5.walmartimages.com/asr/34b610e7-05db-490d-bcc0-80c49addc55b_1.00c3ccc68683f75d9cc0021b3a79d01d.jpeg,5.0
4716,"Versace Man Eau Fraiche Eau De Toilette Spray, Cologne for Men, 1 Oz",24.0,versace,https://i5.walmartimages.com/asr/edaaeed5-9da0-48b8-861f-50165d5fd930_1.3c03e869a55226a4718f313c00f76071.png | https://i5.walmartimages.com/asr/e57d2d1a-cf1d-4c4f-9721-3d83af729eaf_1.51af97bdaa2a960be97936a8011931fb.jpeg,5.0
2058,Goldwell StyleSign 1 Flat Marvel Straightening Balm for Unisex 3.3 oz,2.0,goldwell,https://i5.walmartimages.com/asr/3bf90289-6980-4545-8405-71a08856cd89_1.53ece55389c0533a22773476c0c48841.jpeg,5.0
3842,"Red Devil 0322 Steel Wool # 00 Very Fine, 8 Pack Bag",1.0,"red, devil",https://i5.walmartimages.com/asr/60bfe5ba-774c-40bc-9cb7-42d4fd291767_1.9e569a7fbef1bd802ceb8b175db6e4ec.jpeg | https://i5.walmartimages.com/asr/0daa7bee-ea46-4202-bce2-95ace6787949_1.8ab55949465d49c29e3e1b9bbf36ed29.jpeg | https://i5.walmartimages.com/asr/d0da4c6b-6aab-44cd-9ce6-e16d831f81ad_1.174b7188471bd5f0fe0212de94674d9f.jpeg,5.0
510,"Air Wick Plug in Starter Kit, Warmer + 1 Refill, Pumpkin Spice, Scented Oil, Air Freshener, Essential Oils",1.0,"air, wick",https://i5.walmartimages.com/asr/0fac65b2-c6aa-442f-8952-4d707fef499c.131f9d6b8fc8e3c8bd7fe36bc7974a77.jpeg | https://i5.walmartimages.com/asr/5b494737-e35f-4417-9cd6-11ba9a494a72_1.356fa332e0010f54590538d4bac64566.jpeg | https://i5.walmartimages.com/asr/cd404af3-4198-463e-8b45-5920233d4220_1.3ad427e481279750c6904477d5fa3157.jpeg | https://i5.walmartimages.com/asr/9a093fb5-0fee-4e44-9dec-ef84e37ad47a_1.4d630fb3f617be8ef703b6381ea1d036.png | https://i5.walmartimages.com/asr/978dc0f5-f833-4146-9a08-ca9c2ce9c8e8_1.13cba39da04fb9e6ffdf81701909bb1e.jpeg | https://i5.walmartimages.com/asr/bc010191-5156-473d-aa48-f6e8f11020b6_1.064462d2724b53435feea45aad25e9f8.jpeg | https://i5.walmartimages.com/asr/8edba07d-018a-4487-9668-55b3c36096b0_1.3ac8aa9346d0446c8a70a6d4835d01ee.png | https://i5.walmartimages.com/asr/70807d7d-a82f-4483-aa58-8df4a7acdb11_1.2e7a0be154ecac966631f0c84bdc3556.jpeg,5.0
3841,"Recovery Complex Anti-Frizz Shine Serum by Bain de Terre for Unisex, 4.2 oz",4.0,"bain, de, terre",https://i5.walmartimages.com/asr/fcdb4d2e-3727-4bc4-bb2a-63c585c236b0_1.4c8c7111e5dde79bad7e54b6f71a8781.jpeg | https://i5.walmartimages.com/asr/1e8c5630-bd1b-42bf-b29b-32873

In [61]:
rating_base_recommendation['Rating'] = rating_base_recommendation['Rating'].astype(int)
rating_base_recommendation['ReviewCount'] = rating_base_recommendation['ReviewCount'].astype(int)

<ipython-input-61-14a31ed65ba8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_base_recommendation['Rating'] = rating_base_recommendation['Rating'].astype(int)
<ipython-input-61-14a31ed65ba8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_base_recommendation['ReviewCount'] = rating_base_recommendation['ReviewCount'].astype(int)


In [62]:
rating_base_recommendation

,Name,ReviewCount,Brand,ImageURL,Rating
1686,"Electric Shaver, Triple Shaving Time Electric Rotary Razor for Wet Dry Shaving Waterproof Cordless with Pop-up Trimmer USB Rechargeable",4,moosoo,https://i5.walmartimages.com/asr/e7dcd553-90df-4cf8-8509-bc8035673838.f4f6365105fe9eb0af1bccab5d54b212.jpeg | https://i5.walmartimages.com/asr/5eb0db2d-d3cf-48ba-9017-67bf26d2600f.8d0dd42287deaf6d2b999aa842161d0e.jpeg | https://i5.walmartimages.com/asr/49f1fbb4-b36d-4032-bdf2-9d8402d5ec4d.14532a165720504c8c1d102920bced28.jpeg | https://i5.walmartimages.com/asr/70c133b3-8386-4322-9513-315b33ec21b4.f52654b1dcdd657c6fc6b505c3783a71.jpeg | https://i5.walmartimages.com/asr/b5a354b4-6b28-4cf9-95c5-13a95ea05a62.07b544b8fb25ad4f835d91e69a85daa1.jpeg | https://i5.walmartimages.com/asr/971d2944-630e-4f01-b854-64c121d77973.2edef59cf6780c2f978a33806315662e.jpeg | https://i5.walmartimages.com/asr/80b8528c-08a6-44bd-af75-3aaaa82ed233.d9b9c9a273ad3fe373e15ac16fab5a9d.jpeg | https://i5.walmartimages.com/asr/c4ef66b8-bc76-482e-9a01-82a5317b59be.1ba2aa4f09cf81b14047d6e88b73f51d.jpeg | https://i5.walmartimages.com/asr/6fe4642d-1c1e-4d93-977d-93db17bc37b4.ce7bcefc6486ad1739613c9948c70382.png | https://i5.walmartimages.com/asr/1574b3de-51ba-4146-b65d-c42df0f4478e.5da6e7b5df73f83c7303a25ff2bde2de.jpeg,5
526,"Alaffia Body Lotion, Vanilla, 32 Oz",2,alaffia,https://i5.walmartimages.com/asr/2988c323-cb6f-4a45-9bd7-9029d981630c_1.d65b6410f1b5a72233cdab07e25e153b.jpeg | https://i5.walmartimages.com/asr/c773f069-a99d-4d06-a4ff-d4c839063929_1.3a72dc9092da4efd57530dc86a077c6d.jpeg,5
2053,"Gold Bond Ultimate Ultimate Healing Lotion, Aloe 1 oz",2,"gold, bond",https://i5.walmartimages.com/asr/34b610e7-05db-490d-bcc0-80c49addc55b_1.00c3ccc68683f75d9cc0021b3a79d01d.jpeg,5
4716,"Versace Man Eau Fraiche Eau De Toilette Spray, Cologne for Men, 1 Oz",24,versace,https://i5.walmartimages.com/asr/edaaeed5-9da0-48b8-861f-50165d5fd930_1.3c03e869a55226a4718f313c00f76071.png | https://i5.walmartimages.com/asr/e57d2d1a-cf1d-4c4f-9721-3d83af729eaf_1.51af97bdaa2a960be97936a8011931fb.jpeg,5
2058,Goldwell StyleSign 1 Flat Marvel Straightening Balm for Unisex 3.3 oz,2,goldwell,https://i5.walmartimages.com/asr/3bf90289-6980-4545-8405-71a08856cd89_1.53ece55389c0533a22773476c0c48841.jpeg,5
3842,"Red Devil 0322 Steel Wool # 00 Very Fine, 8 Pack Bag",1,"red, devil",https://i5.walmartimages.com/asr/60bfe5ba-774c-40bc-9cb7-42d4fd291767_1.9e569a7fbef1bd802ceb8b175db6e4ec.jpeg | https://i5.walmartimages.com/asr/0daa7bee-ea46-4202-bce2-95ace6787949_1.8ab55949465d49c29e3e1b9bbf36ed29.jpeg | https://i5.walmartimages.com/asr/d0da4c6b-6aab-44cd-9ce6-e16d831f81ad_1.174b7188471bd5f0fe0212de94674d9f.jpeg,5
510,"Air Wick Plug in Starter Kit, Warmer + 1 Refill, Pumpkin Spice, Scented Oil, Air Freshener, Essential Oils",1,"air, wick",https://i5.walmartimages.com/asr/0fac65b2-c6aa-442f-8952-4d707fef499c.131f9d6b8fc8e3c8bd7fe36bc7974a77.jpeg | https://i5.walmartimages.com/asr/5b494737-e35f-4417-9cd6-11ba9a494a72_1.356fa332e0010f54590538d4bac64566.jpeg | https://i5.walmartimages.com/asr/cd404af3-4198-463e-8b45-5920233d4220_1.3ad427e481279750c6904477d5fa3157.jpeg | https://i5.walmartimages.com/asr/9a093fb5-0fee-4e44-9dec-ef84e37ad47a_1.4d630fb3f617be8ef703b6381ea1d036.png | https://i5.walmartimages.com/asr/978dc0f5-f833-4146-9a08-ca9c2ce9c8e8_1.13cba39da04fb9e6ffdf81701909bb1e.jpeg | https://i5.walmartimages.com/asr/bc010191-5156-473d-aa48-f6e8f11020b6_1.064462d2724b53435feea45aad25e9f8.jpeg | https://i5.walmartimages.com/asr/8edba07d-018a-4487-9668-55b3c36096b0_1.3ac8aa9346d0446c8a70a6d4835d01ee.png | https://i5.walmartimages.com/asr/70807d7d-a82f-4483-aa58-8df4a7acdb11_1.2e7a0be154ecac966631f0c84bdc3556.jpeg,5
3841,"Recovery Complex Anti-Frizz Shine Serum by Bain de Terre for Unisex, 4.2 oz",4,"bain, de, terre",https://i5.walmartimages.com/asr/fcdb4d2e-3727-4bc4-bb2a-63c585c236b0_1.4c8c7111e5dde79bad7e54b6f71a8781.jpeg | https://i5.walmartimages.com/asr/1e8c5630-bd1b-42bf-b29b-328734e30ea1_1.1c9ebfea6bdb2db875f2

  Content-based recommendation

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
pd.set_option('display.max_colwidth', None)
df['Tags'].head(5)

,Tags
0,"premium, beauty, premium, makeup, premium, nail, polish, care, premium, nail, polish, opi,"
1,"beauty, hair, care, hair, color, auburn, hair, color, easy, pack, 3, pack, 3, upc, 381519000201, beautiful, natural, color, active, ingredients, color, blend, formula, water, ethoxydiglycol, propylene, glycol, isopropyl, alcohol, soytrimonium, chloride, oleic, acid, ammonium, hydroxide, trisodium, ethylenediamine, disuccinate, fragrance, c11, 15, cocamidopropyl, betaine, c12, 15, erythorbic, acid, citric, acid, sodium, sulfite, edta, p, aminophenol, 4, hydroxytoluene, p, phenylenediamine, resorcinol, 2, methylresorcinol, n, n, phenylenediamine, sulfate, 1, hydroxyethyl, diamino, pyrazole, sulfate, phenyl, methyl, pyrazolone, 1, naphthol, color, blend, activator, water, hydrogen, peroxide, acrylates, copolymer, hydrogenated, palmamide, acrylates, methacrylate, copolymer, oleyl, alcohol, etidronic, acid, disodium, edta, simethicone, colorseal, conditioning, gloss, water, amodimethicone, stearyl, alcohol, cetyl, alcohol, stearamidopropyl, dimethylamine, glutamic, acid, fragrance, benzyl, alcohol, panthenyl, ethyl, ether, edta, panthenol, trimethylsiloxysilicate, carthamus, tinctorius, safflower, seed, oil, cocos, nucifera, coconut, oil, hydrolyzed, sweet, almond, protein, methylchloroisothiazolinone, methylisothiazolinone, aloe, barbadensis, leaf, uses, salon, experts, know, takes, perfect, blend, tones, create, amazingly, natural, color, results, nice, n, easy, creates, beautifully, blended, natural, looking, shades, like, experts, easy, step, know, nice, n, easy, right, nice, n, easy, gives, ultimate, natural, looking, color, highlights, shine, easy, step, browns, brilliant, blondes, illuminating, reds, radiant, work, nice, n, easy, exclusive, color, blend, formula, step, shade, gives, expert, blend, 3, tones, work, create, natural, looking, color, shining, dimension, cover, grays, yes, nice, n, easy, promises, 100, gray, coverage, resistant, grays, long, nice, n, easy, lasts, 6, weeks, radiant, red, shade, ll, find, 6, weekly, treatments, exclusive, colorseal, conditioning, gloss, seal, healthy, looking, color, add, intense, shine"
2,"beauty, hair, care, hair, color, permanent, hair, color, clairol, clairol, nice, n, easy, permanent, color, gives, 8, weeks, true, color, salon, tones, blend, highlights, lowlights, expertly, designed, work, unique, hair, stylish, look, want, neutral, blonde, color, offers, salon, quality, permanent, hair, color, cost, easily, color, looking, vibrant, clairol, colorseal, conditioner, locks, softness, shine, sold, separately, cover, grays, colorings, try, nice, n, easy, root, touch, blend, seamlessly, color, natural, dark, hair, color, easy, apply, fresh, looking, color, right, comfort, home, comes, 1, kit, providing, need, color, hair, desired, available, shades, clairol, nice, n, easy, permanent, color, lovely, addition, haircare, routine, clairol, nice, n, easy, permanent, color, natural, dark, neutral, blonde, kit, provides, 8, weeks, color, 3, salon, tones, offer, blend, highlights, lowlights, salon, quality, permanent, color, neutral, blonde, color, easy, use, available, ultra, light, natural, blo, 118, natural, medium, 2, natural, black, soft, black, natural, blue, black, natural, darkest, brown, natural, dark, brown, natural, dark, natural, dark, auburn, dark, caramel, brown"
3,"beauty, makeup, lip, kokie, cosmetics, calling, matte, lip, lovers, indulge, rich, creamy, matte, lipsticks, bold, long, lasting, opaque, babies, pack, punch, pair, lip, liners, extra, definition, lip, primer, longer, wear, ahead, grab, matte, lip, professional, sienna, matte, lipstick, highly, pigmented, rich, long, wear, matte, lipstick, intense, color, high, payoff, color, smooth, matte, application"
4,"seasonal, stock, essentials, personal, care, stock, stock, shave, gillette, 1971, gillette, introduced, trac, ii, razor, system, trac, ii, razor, fixed, head, twin, blade, shaving, system, kind, north, america,

In [65]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix_content = tfidf_vectorizer.fit_transform(df['Tags'])
cosine_similarities_content = cosine_similarity(tfidf_matrix_content,tfidf_matrix_content)

In [66]:
cosine_similarities_content[1]

array([0.00347577, 1.        , 0.38086046, ..., 0.02315363, 0.00183907,
       0.0381051 ])

In [67]:
item_name = 'Kokie Professional Matte Lipstick, Hot Berry, 0.14 fl oz'
item_index = df[df['Name']==item_name].index[0]

In [68]:
item_index

3

In [69]:
similar_items = list(enumerate(cosine_similarities_content[item_index]))
similar_items

[(0, 0.013151377769582695),
 (1, 0.04040483010081997),
 (2, 0.047427471122117024),
 (3, 1.0000000000000002),
 (4, 0.0019674534994322413),
 (5, 0.10177577820210486),
 (6, 0.0),
 (7, 0.020589528021892112),
 (8, 0.014694826317389335),
 (9, 0.20726465762721494),
 (10, 0.09063286003702592),
 (11, 0.001045153206069322),
 (12, 0.04015685993534293),
 (13, 0.014755178024583062),
 (14, 0.0193361372460227),
 (15, 0.02504115552161843),
 (16, 0.004206031711074756),
 (17, 0.0),
 (18, 0.20885160306697956),
 (19, 0.01034671937341378),
 (20, 0.07501343963557214),
 (21, 0.0017612414141947655),
 (22, 0.0),
 (23, 0.008455891578228824),
 (24, 0.0),
 (25, 0.0),
 (26, 0.007861554782979988),
 (27, 0.006316125733980389),
 (28, 0.0034799224363695497),
 (29, 0.0),
 (30, 0.020747727414955106),
 (31, 0.006827634205025217),
 (32, 0.0),
 (33, 0.0027603821499158254),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.022461560263817668),
 (38, 0.004556157359523357),
 (39, 0.02313476542984557),
 (40, 0.0006295185474606853),


In [70]:
similar_items = sorted(similar_items, key=lambda x:x[1], reverse=True)
similar_items

[(3, 1.0000000000000002),
 (3406, 1.0000000000000002),
 (546, 0.6588361533659113),
 (2406, 0.5584693534410101),
 (4050, 0.5392146445343637),
 (4084, 0.5094443493526426),
 (1559, 0.5038552685536598),
 (2873, 0.4838014504767311),
 (3023, 0.4554308275017389),
 (4541, 0.4518901227878721),
 (1315, 0.44529262566642575),
 (1300, 0.4266143864559126),
 (100, 0.4203359016801316),
 (4872, 0.41779467300219236),
 (868, 0.40946471882855945),
 (2557, 0.38284979373682726),
 (2051, 0.3800327606019105),
 (4619, 0.37554419068382977),
 (1049, 0.37123334704864086),
 (181, 0.36943543125618006),
 (938, 0.36943543125618006),
 (1109, 0.36913923222212974),
 (2059, 0.3671440109432047),
 (2480, 0.3671225495747757),
 (838, 0.36563245871248456),
 (4640, 0.36252150537548494),
 (3502, 0.36152254021730695),
 (420, 0.354931716442042),
 (2032, 0.3546807886929964),
 (1313, 0.35128284239879193),
 (780, 0.3473047663862309),
 (1121, 0.3451288779004318),
 (464, 0.3371763767258313),
 (2838, 0.3322807500110498),
 (379, 0.32925

In [71]:
similar_items = sorted(similar_items, key=lambda x:x[1], reverse=True)
top_similar_items = similar_items[1:10]
recommended_items_indices = [x[0] for x in top_similar_items]
recommended_items_indices #got top similar index to recommend

[3406, 546, 2406, 4050, 4084, 1559, 2873, 3023, 4541]

In [72]:
result = df[['Name', 'ReviewCount', 'Brand']].take(recommended_items_indices)
result

,Name,ReviewCount,Brand
3406,"Kokie Professional Matte Lipstick, Firecracker, 0.14 fl oz",0.0,"kokie, cosmetics"
546,"Kokie Professional Matte Lipstick, Kiss Me, 0.14 fl oz",0.0,"kokie, cosmetics"
2406,"L.A. Colors Matte Lipstick, Tender Matte",3.0,colors
4050,"Kokie Professional Lip Poudre Liquid Matte Liquid Lipstick, Infamous",7.0,"kokie, cosmetics"
4084,e.l.f. Mad for Matte 4 Piece Lip Color Set,0.0,cosmetics
1559,"LOreal Paris Colour Riche Matte Lip Liner, Mattes It, 0.04 oz.",495.0,paris
2873,"Kokie Professional Lip Poudre Liquid Matte Liquid Lipstick, Charmed",7.0,"kokie, cosmetics"
3023,Be Matte Lipstick - Pink,2.0,"city, color"
4541,Be Matte Lipstick - Soft Pink,2.0,"city, color"


Combining the whole step to make recommendation

In [73]:
def content_based_recommendations(train_data, item_name, top_n=10):
    if item_name not in train_data['Name'].values:
        print(f"Item '{item_name}' not found in the training data.")
        return pd.DataFrame()
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix_content = tfidf_vectorizer.fit_transform(train_data['Tags'])
    cosine_similarities_content = cosine_similarity(tfidf_matrix_content, tfidf_matrix_content)
    item_index = df[train_data['Name'] == item_name].index[0]
    similar_items = list(enumerate(cosine_similarities_content[item_index]))
    similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)
    top_similar_items = similar_items[1:top_n+1]
    recommended_item_indices = [x[0] for x in top_similar_items]
    recommended_items_details = df[['Name', 'ReviewCount', 'Brand', 'ImageURL', 'Rating']].take(recommended_items_indices)
    return recommended_items_details

In [74]:
item_name = 'OPI Infinite Shine, Nail Lacquer Nail Polish, Bubble Bath'
content_based_recommendations(df, item_name)

,Name,ReviewCount,Brand,ImageURL,Rating
3406,"Kokie Professional Matte Lipstick, Firecracker, 0.14 fl oz",0.0,"kokie, cosmetics",https://i5.walmartimages.com/asr/8312221b-ed22-4b22-890b-e6c5779ce493_1.16df0fa24be16937a7a592f5f535c2f1.jpeg,0.0
546,"Kokie Professional Matte Lipstick, Kiss Me, 0.14 fl oz",0.0,"kokie, cosmetics",https://i5.walmartimages.com/asr/27dd82a2-2b9c-4a09-bb02-4416c7cb13b7_1.a88c173f67d959f76070e3251f6c0a7b.jpeg | https://i5.walmartimages.com/asr/c4ba023a-5195-4277-ae57-022efdc3746c_1.bdffbf8987fee7e90f41f747a2c5222d.jpeg,0.0
2406,"L.A. Colors Matte Lipstick, Tender Matte",3.0,colors,https://i5.walmartimages.com/asr/271264fb-e8c3-4089-80d0-d08b72a1eca5_1.67b0f6a2a615c366e5d96257ba32ac7b.jpeg,3.7
4050,"Kokie Professional Lip Poudre Liquid Matte Liquid Lipstick, Infamous",7.0,"kokie, cosmetics",https://i5.walmartimages.com/asr/fdd7498c-319f-4791-bc1f-fdea6f6ead4e_1.0a97822ba508366103e5533a1e2d71b9.jpeg | https://i5.walmartimages.com/asr/f21043f6-6616-4006-948e-ea8f366c27df_2.789b8e6718aa2396b8c25ee101a6f4ba.jpeg,3.4
4084,e.l.f. Mad for Matte 4 Piece Lip Color Set,0.0,cosmetics,https://i5.walmartimages.com/asr/e2d30304-edc9-4bd2-91ff-f1dbc7984f40_1.11ebe603203c5e9060f35506803c5737.jpeg | https://i5.walmartimages.com/asr/3f0505f8-2c82-48b4-916d-434220931ea3_1.0eccb29b769e361dde2d0bfd59702f26.jpeg,0.0
1559,"LOreal Paris Colour Riche Matte Lip Liner, Mattes It, 0.04 oz.",495.0,paris,https://i5.walmartimages.com/asr/baf97085-7231-4974-8225-272aaa25a734.c9651ae6dc5b76ede55d0968dbe37624.jpeg | https://i5.walmartimages.com/asr/677c7520-baf4-4b05-846c-55a0fdbe9045.9e52072e7d45175511101bf1b62aec59.jpeg,4.4
2873,"Kokie Professional Lip Poudre Liquid Matte Liquid Lipstick, Charmed",7.0,"kokie, cosmetics",https://i5.walmartimages.com/asr/31c99d9b-ea11-449d-9235-1c09bd5522dc_1.e48c97350e1e23c34975d833bbca0f1b.jpeg | https://i5.walmartimages.com/asr/8f31120e-956d-4cc8-908a-d63bd6864746_2.789b8e6718aa2396b8c25ee101a6f4ba.jpeg,3.4
3023,Be Matte Lipstick - Pink,2.0,"city, color",https://i5.walmartimages.com/asr/4425a13e-085f-4189-8d2c-16652f1065d8_1.2fdd758908c76273f92a428b8f9e832c.jpeg,3.0
4541,Be Matte Lipstick - Soft Pink,2.0,"city, color",https://i5.walmartimages.com/asr/69e1ec63-be5e-4f08-8da5-1a5fb2153db2_1.44e735209e5fad2c6e72e5cb00818c54.jpeg,3.0
